# Upload sequencing data to SRA
This Python Jupyter notebook uploads the sequencing data to the NIH [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra), or SRA.

## Create BioProject and BioSamples
The first step was done manually to create the BioProject and BioSamples.
Note that for new future uploads related to the RBD DMS, you may be able to use the existing BioProject, but since this is the first entries in these project I needed to create a new BioProject.

To create these, I went to the [Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra) and signed in using the box at the upper right of the webpage, and then went to the [SRA Submission Portal](https://submit.ncbi.nlm.nih.gov/subs/sra/).
I then manually completed the first five steps, which define the project and samples.

## Create submission sheet
The sixth step is to create the submission sheet in `*.tsv` format, which is done by the following code.

First, import Python modules:

In [1]:
import ftplib
import os
import tarfile
import datetime
import glob
import itertools

import natsort

import pandas as pd

import yaml

Read the configuration for the analysis:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

Read the PacBio runs:

In [3]:
pacbio_runs_file = os.path.join('../', config['pacbio_runs'])

print(f"Reading PacBio runs from {pacbio_runs_file}")

pacbio_runs = (
    pd.read_csv(pacbio_runs_file)
#     .assign(ccs_file=lambda x: f"../{config['ccs_dir']}/" + x['library'] + '_' + x['run'] + '_ccs.fastq.gz')
    .assign(ccs_file=lambda x: x['ccs'])
    )

pacbio_runs.head()

Reading PacBio runs from ../data/PacBio_runs.csv


,library,run,ccs,ccs_file
0,lib1,210701_A,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...
1,lib2,210701_A,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...


Next make submission entries for the PacBio CCSs:

In [4]:
pacbio_submissions = (
    pacbio_runs
    .assign(
        sample_name='PacBio_CCSs',  # BioSample created in SRA wizard
        library_ID=lambda x: 'B1351_' + x['library'] + '_PacBio_CCSs',  # unique library ID
        title='PacBio CCSs linking variants to barcodes for SARS-CoV-2 B.1.351 RBD deep mutational scanning',
        library_strategy='Synthetic-Long-Read',
        library_source='SYNTHETIC',
        library_selection='Restriction Digest',
        library_layout='single',
        platform='PACBIO_SMRT',
        instrument_model='PacBio Sequel II',
        design_description='Restriction digest of plasmids carrying barcoded RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['ccs_file'],      
        )
    .drop(columns=pacbio_runs.columns)
    )

pacbio_submissions.head()

,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath
0,PacBio_CCSs,B1351_lib1_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...
1,PacBio_CCSs,B1351_lib2_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...


Read the Illumina runs:

In [5]:
illumina_runs_file = os.path.join('../', config['barcode_runs'])

print(f"Reading Illumina runs from {illumina_runs_file}")

illumina_runs = pd.read_csv(illumina_runs_file)

illumina_runs.head()

Reading Illumina runs from ../data/barcode_runs.csv


,date,experiment,library,antibody,concentration,sort_bin,selection,sample,experiment_type,number_cells,frac_escape,R1
0,210622,bg4e_6-9,lib1,none,0,ref,reference,bg4e_6-9-none-0-ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/210628_D00300_12...
1,210622,bg4e_6-9,lib2,none,0,ref,reference,bg4e_6-9-none-0-ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/210628_D00300_12...
2,210622,bg4e_8,lib1,K115_old,80,abneg,escape,bg4e_8-K115_old-80-abneg,ab_selection,604025.0,0.036,/shared/ngs/illumina/agreaney/210628_D00300_12...
3,210622,bg4e_8,lib2,K115_old,80,abneg,escape,bg4e_8-K115_old-80-abneg,ab_selection,566545.0,0.034,/shared/ngs/illumina/agreaney/210628_D00300_12...
4,210702,bg4e_10,lib1,K007,500,abneg,escape,bg4e_10-K007-500-abneg,ab_selection,510107.0,0.038,/shared/ngs/illumina/agreaney/210702_D00300_12...


Next make submission entries for Illumina data:

In [6]:
illumina_submissions = (
    illumina_runs
    .query('experiment not in ["bg4e_6-9", "bg4e_8"]')
    .assign(
        sample_name=lambda x: 'B1351_plasma_escape_barcodes',
        library_ID=lambda x: 'B1351_' + x['library'] + '_' + x['sample'],
        title=lambda x: 'Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename)
           )
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

illumina_submissions.head()

,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_10_lib1_abneg_S9_R1_001.fastq.gz
1,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_10_lib2_abneg_S10_R1_001.fastq.gz
2,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_11-K031-500-abneg,Illumina barcode sequencing for bg4e_11-K031-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_11_lib1_abneg_S11_R1_001.fastq.gz
3,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_11-K031-500-abneg,Illumina barcode sequencing for bg4e_11-K031-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_11_lib2_abneg_S12_R1_001.fastq.gz
4,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_12-K033-500-abneg,Illumina barcode sequencing for bg4e_12-K033-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_12_lib1_abneg_S13_R1_001.fastq.gz


Now concatenate the PacBio and Illumina submissions into tidy format (one line per file), make sure all the files exist, and also make short name versions of them that lack the path:

In [7]:
submissions_tidy = (
    pd.concat([pacbio_submissions, illumina_submissions], ignore_index=True)
    .assign(file_exists=lambda x: x['filename_fullpath'].map(os.path.isfile),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename),
            )
    )

assert submissions_tidy['file_exists'].all(), submissions_tidy.query('file_exists == False')

submissions_tidy.head()

,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename,file_exists
0,PacBio_CCSs,B1351_lib1_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...,demultiplex.bc1021_BAK8B_OA--bc1021_BAK8B_OA.h...,True
1,PacBio_CCSs,B1351_lib2_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,/fh/fast/bloom_j/SR/ngs/pacbio_UW/20210701_Gre...,demultiplex.bc1022_BAK8B_OA--bc1022_BAK8B_OA.h...,True
2,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_10_lib1_abneg_S9_R1_001.fastq.gz,True
3,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_10_lib2_abneg_S10_R1_001.fastq.gz,True
4,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_11-K031-500-abneg,Illumina barcode sequencing for bg4e_11-K031-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/210702_D00300_12...,bg4e_11_lib1_abneg_S11_R1_001.fastq.gz,True


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [8]:
(submissions_tidy
 .groupby(['sample_name', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,sample_name,library_ID,n_files
0,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_10-14-reference-0-ref,4
1,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_15-18-reference-0-ref,4
2,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_15-18-reference-0-ref,4
3,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_10-14-reference-0-ref,4
4,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_11-K031-500-abneg,1
5,PacBio_CCSs,B1351_lib1_PacBio_CCSs,1
6,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_18-K119-200-abneg,1
7,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_17-K115-80-abneg,1
8,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_16-K114-200-abneg,1
9,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_15-K046-200-abneg,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [9]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['sample_name', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'file_exists', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

submissions_wide

,sample_name,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3,filename4
0,PacBio_CCSs,B1351_lib1_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,demultiplex.bc1021_BAK8B_OA--bc1021_BAK8B_OA.h...,NaN,NaN,NaN
1,PacBio_CCSs,B1351_lib2_PacBio_CCSs,PacBio CCSs linking variants to barcodes for S...,Synthetic-Long-Read,SYNTHETIC,Restriction Digest,single,PACBIO_SMRT,PacBio Sequel II,Restriction digest of plasmids carrying barcod...,fastq,demultiplex.bc1022_BAK8B_OA--bc1022_BAK8B_OA.h...,NaN,NaN,NaN
2,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_10_lib1_abneg_S9_R1_001.fastq.gz,NaN,NaN,NaN
3,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_10-K007-500-abneg,Illumina barcode sequencing for bg4e_10-K007-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_10_lib2_abneg_S10_R1_001.fastq.gz,NaN,NaN,NaN
4,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_11-K031-500-abneg,Illumina barcode sequencing for bg4e_11-K031-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_11_lib1_abneg_S11_R1_001.fastq.gz,NaN,NaN,NaN
5,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_11-K031-500-abneg,Illumina barcode sequencing for bg4e_11-K031-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_11_lib2_abneg_S12_R1_001.fastq.gz,NaN,NaN,NaN
6,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_12-K033-500-abneg,Illumina barcode sequencing for bg4e_12-K033-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_12_lib1_abneg_S13_R1_001.fastq.gz,NaN,NaN,NaN
7,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_12-K033-500-abneg,Illumina barcode sequencing for bg4e_12-K033-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_12_lib2_abneg_S14_R1_001.fastq.gz,NaN,NaN,NaN
8,B1351_plasma_escape_barcodes,B1351_lib1_bg4e_13-K040-500-abneg,Illumina barcode sequencing for bg4e_13-K040-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_13_lib1_abneg_S15_R1_001.fastq.gz,NaN,NaN,NaN
9,B1351_plasma_escape_barcodes,B1351_lib2_bg4e_13-K040-500-abneg,Illumina barcode sequencing for bg4e_13-K040-5...,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,bg4e_13_lib2_abneg_S16_R1_001.fastq.gz,NaN,NaN,NaN


Now write the wide submissions data frame to a `*.tsv` file:

In [10]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

This submission sheet was then manually uploaded in Step 6 of the SRA submission wizard (*SRA metadata*).

## Upload the actual files
Step 7 of the SRA submission wizard is to upload the files.
In order to do this, we first make a `*.tar` file with all of the files.
Since this takes a long time, we only create the file if it doesn't already exist, so it is only created the first time this notebook is run.
**Note that this will cause a problem if you add more sequencing files to upload after running the notebook, in that case the cell below will need to altered.**

In [11]:
tar_filename = 'SRA_submission.tar'

if os.path.isfile(tar_filename):
    print(f"{tar_filename} already exists, not creating it again")
else:
    try:
        with tarfile.open(tar_filename, mode='w') as f:
            for i, tup in enumerate(submissions_tidy.itertuples()):
                print(f"Adding file {i + 1} of {len(submissions_tidy)} to {tar_filename}")
                f.add(tup.filename_fullpath, arcname=tup.filename)
            print(f"Added all files to {tar_filename}")
    except:
        if os.path.isfile(tar_filename):
            os.remove(tar_filename)
        raise

Adding file 1 of 36 to SRA_submission.tar
Adding file 2 of 36 to SRA_submission.tar
Adding file 3 of 36 to SRA_submission.tar
Adding file 4 of 36 to SRA_submission.tar
Adding file 5 of 36 to SRA_submission.tar
Adding file 6 of 36 to SRA_submission.tar
Adding file 7 of 36 to SRA_submission.tar
Adding file 8 of 36 to SRA_submission.tar
Adding file 9 of 36 to SRA_submission.tar
Adding file 10 of 36 to SRA_submission.tar
Adding file 11 of 36 to SRA_submission.tar
Adding file 12 of 36 to SRA_submission.tar
Adding file 13 of 36 to SRA_submission.tar
Adding file 14 of 36 to SRA_submission.tar
Adding file 15 of 36 to SRA_submission.tar
Adding file 16 of 36 to SRA_submission.tar
Adding file 17 of 36 to SRA_submission.tar
Adding file 18 of 36 to SRA_submission.tar
Adding file 19 of 36 to SRA_submission.tar
Adding file 20 of 36 to SRA_submission.tar
Adding file 21 of 36 to SRA_submission.tar
Adding file 22 of 36 to SRA_submission.tar
Adding file 23 of 36 to SRA_submission.tar
Adding file 24 of 36

See the size of the `*.tar` file to upload and make sure it has the expected files:

In [12]:
print(f"The size of {tar_filename} is {os.path.getsize(tar_filename) / 1e9:.1f} GB")

with tarfile.open(tar_filename) as f:
    files_in_tar = set(f.getnames())
if files_in_tar == set(submissions_tidy['filename']):
    print(f"{tar_filename} contains all {len(files_in_tar)} expected files.")
else:
    raise ValueError(f"{tar_filename} does not have all the expected files.")

The size of SRA_submission.tar is 15.9 GB
SRA_submission.tar contains all 36 expected files.


The SRA instructions then give several ways to upload; we will do it using the FTP method.
First, specify the FTP address, username, password, and subfolder given by the SRA submission wizard instructions.
In order to avoid having the password be public here, that is in a separate text file that is **not** included in the GitHub repo (so this needs to be run in Jesse's directory that has this password):

In [13]:
# the following are provided by SRA wizard insturctions
ftp_address = 'ftp-private.ncbi.nlm.nih.gov'
ftp_username = 'subftp'
ftp_account_folder = 'uploads/agreaney_uw.edu_hNzBp9b0'
with open('ftp_password.txt') as f:
    ftp_password = f.read().strip()
    
# meaningful name for subfolder
ftp_subfolder = 'SARS-CoV-2-RBD_B.1.351_try2'

Now create FTP connection and upload the TAR file.
Note that this takes a while.
If you are worried that it will timeout given the size of your file, you can run this notebook via `slurm` so there is no timing out:

In [14]:
print(f"Starting upload at {datetime.datetime.now()}")

with ftplib.FTP(ftp_address) as ftp:
    ftp.login(user=ftp_username,
              passwd=ftp_password,
              )
    ftp.cwd(ftp_account_folder)
    ftp.mkd(ftp_subfolder)
    ftp.cwd(ftp_subfolder)
    with open(tar_filename, 'rb') as f:
        ftp.storbinary(f"STOR {tar_filename}", f)
        
print(f"Finished upload at {datetime.datetime.now()}")

Starting upload at 2021-10-10 10:49:53.206295
Finished upload at 2021-10-10 13:33:12.633022


Finally, used the SRA wizard to select the `*.tar` archive and complete the submission.
Note that there is a warning of missing files since everything was uploaded as a `*.tar` rather than individual files.
They should all be found when you hit the button to proceed and the `*.tar` is unpacked.

There was then a message that the submission was processing, and data would be released immediately upon processing.
The submission number is `SUB10502912`.